## 对象引用、可变性和垃圾回收

## 变量不是装值的盒子
- 一般会说把对象分配给变量，而不是把变量分配给对象
- 变量是到具体值的引用，创建对象之后，值已经存在，之后才把变量分配到对象，创建到值的引用
- 对于赋值语句，对象先在右边创建或者获取，之后左边的变量才会绑定到对象上。变相就像是对象上贴的一个标注，一个对象可能会被贴多个标注（而不可能同时在多个盒子里）

## 在下面的例子中，a和b是别名，贴在同一个对象上；c和a,b的值一样，但贴在不同的对象上

In [23]:
import doctest

In [17]:
a = {'name': 'charles'}
b = a
a is b

True

In [18]:
c = {'name': 'charles'}
c == a

True

In [19]:
a is c

False

In [20]:
print(id(a), id(b), id(c))

2596893231000 2596893231000 2596893330240


## 元组的不可变性指的是元组数据结构的物理内容（即保存的引用）不可变，与引用的对象无关
- 元组的值会随着引用的可变对象的变化而变
- 不可变的是元素的标识(id)

## 使用构造方法或者[:]进行列表复制是浅复制
- 只复制外层容器
- 副本中的元素是源容器中元素的引用
- **如下，l1和l2指向不同的列表对象，但列表元素中的列表和元组指向相同的对象**
- **同时对于列表和元组，即便它们作为列表元素，也保留了本身特性，即列表做浅复制而元组做深复制**

In [58]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
print(l1 is l2)
print(l1[0] is l2[0])
print(l1[1] is l2[1])
print(l1[2] is l2[2])
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)
l2[1] += [33, 22]
l2[2] += (10, 11)
print(l1[0] is l2[0])
print(l1[1] is l2[1])
print(l1[2] is l2[2])
print('l1:', l1)
print('l2:', l2)

False
True
True
True
l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
True
True
False
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


## 使用deepcopy为任意对象做深复制
- 即副本不共享内部对象的引用
- deepcopy可以处理循环引用

In [25]:
class Bus:
    """
        >>> import copy
        >>> bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
        >>> bus2 = copy.copy(bus1)
        >>> bus3 = copy.deepcopy(bus1)
        >>> bus1.drop('Bill')
        >>> bus2.passengers
        ['Alice', 'Claire', 'David']
        >>> bus3.passengers
        ['Alice', 'Bill', 'Claire', 'David']
    """

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [26]:
doctest.testmod(verbose=True)

Trying:
    import copy
Expecting nothing
ok
Trying:
    bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
Expecting nothing
ok
Trying:
    bus2 = copy.copy(bus1)
Expecting nothing
ok
Trying:
    bus3 = copy.deepcopy(bus1)
Expecting nothing
ok
Trying:
    bus1.drop('Bill')
Expecting nothing
ok
Trying:
    bus2.passengers
Expecting:
    ['Alice', 'Claire', 'David']
ok
Trying:
    bus3.passengers
Expecting:
    ['Alice', 'Bill', 'Claire', 'David']
ok
5 items had no tests:
    __main__
    __main__.Bus.__init__
    __main__.Bus.drop
    __main__.Bus.pick
    __main__.yapf_reformat
1 items passed all tests:
   7 tests in __main__.Bus
7 tests in 6 items.
7 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=7)

In [55]:
import copy
a = [1, 2]
b = [a, 3]
a.append(b)
print(a)
print(a[2])
print(a[2][0][2][0][2][0][2][0][2][0][2][0][2])
b = copy.deepcopy(a)
print(b[2][0][2][0][2][0][2][0][2][0][2][0][2])
print(b is a)

[1, 2, [[...], 3]]
[[1, 2, [...]], 3]
[[1, 2, [...]], 3]
[[1, 2, [...]], 3]
False


## Python只支持共享传参
- 函数的各个形式参数获得实参中各个引用的副本
- 函数内部的形参是实参的别名
- 因此函数可能会修改接收到的任何可变对象

In [39]:
def f(a, b):
    a += b
    return a


x = 1
y = 2
print(f(x, y))
print(x, y)

3
1 2


In [40]:
a = [1, 2]
b = [3, 4]
print(f(a, b))
print(a, b)

[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]


## 可选参数可以有默认值，但是默认值应该避免使用可变的对象，否则在初始化一个新对象时，其参数可能指向一个之前的对象的参数（默认可变对象）
- 比如下面这个幽灵车的例子，bus3初始化后，应该没有任何乘客，但是出现了bus2中的'Carrie'，之后bus3中上车的乘客也出现在了bus2中。这是因为两个bus使用默认值初始化，这时默认值是一个可变列表，因此它们指向同一个列表
- 因此一般用None作为默认值，否则则需要将参数passengers的副本传给self.passengers

In [41]:
class HauntedBus:
    def __init__(self, passengers=[]):
        self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [45]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [46]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [47]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [48]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [49]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [50]:
bus2.passengers is bus3.passengers

True

In [51]:
bus1.passengers

['Bill', 'Charlie']

## 因此当不希望函数修改传入的参数时，应该防御可变参数
- 即做一次浅复制

In [59]:
class TwilightBus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

## 对象绝不会自行销毁，然而，无法得到对象时，可能会被当作垃圾回收
- del删除名称，而不是对象
- 当del删除了对象的最后一个引用时，或者两个对象只剩下相互引用时，对象会被销毁

## 弱引用
- 场景：需要引用对象，但是不让对象存在的时间超过所需时间
- 弱引用不增加对象的引用数量，不妨碍对象被回收，弱引用的目标对象成为所指对象(referent)
- int和tuple实例不能作为弱引用目标
- set可以
- list不可以，但是list的子类可以

In [66]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)
print(wref)

<weakref at 0x0000025CA369CEA8; to 'set' at 0x0000025CA358B588>


In [67]:
wref()

{0, 1}

In [68]:
a_set = {2, 3, 4}

In [69]:
wref()

{0, 1}

In [73]:
print(wref() is None)
print(wref() is None)

False
False


## WeakValueDictionary
- 值是对象的弱引用
- 当被引用的对象被回收后，对应的键会自动删除。该结构多用于缓存
- 对应的还有WeakKeyDictionary、WeakSet

In [75]:
class Cheese:
    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return 'Cheese(%r)' % self.kind

In [76]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [
    Cheese('Red Leicester'),
    Cheese('Tilsit'),
    Cheese('Brie'),
    Cheese('Parmesan')
]

In [78]:
for cheese in catalog:
    stock[cheese.kind] = cheese

In [79]:
sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [80]:
del catalog
sorted(stock.keys())

['Parmesan']

In [81]:
del cheese
sorted(stock.keys())

[]

## 在删除catalog后，只留下了Parmesan，为什么？
- **因为还有一个指向Parmesan的引用，即cheese，Parmesan是cheese最后一次循环指向的对象**